### **Creating A De Bruijn Graph**
##### *Rose Wilfong*

This note book will have 3 main functions: 

1. Generate random DNA sequence 

2. Slice the DNA sequence into kmers of designated length 

3. Plot the kmers in a De Bruijn graph 
    
    kmers are nodes, overlaps are edges

The DNA sequence creator and kmer size will be changeable in the first cell.


**Needed Libraries**: toyplot, numpy, random, textwrap

- Toyplot is used to create the simple line graph

- Random is used to generate random integers between 0 and 3 for the DNA sequence generation
- Textwrap is used to write DNA sequences to a FASTA file  

Link to toyplot installation: https://toyplot.readthedocs.io/en/stable/installation.html 

 
The **coverage** can be calculated as:
- coverage = L*N/G 
- L = length of kmer
- N = number of generated kmers 
- G = total length of the input sequence 

The **error rate** can be calculated as:
- 1-(1*10e-2) per base 


To do: add option to save the resulting graph as a png, add coverage and error rates for dna seq generation

In [106]:
## Inputs
# input for DNA generator
num_dna = 1  #int 
min_length = 20 #int
max_length = 30 #int 
file_name = "sequence1.fa" # file name for saving fasta file 

# add in coverage and error rate if there's time

# input for de Bruijn/kmer generator 
kmer_len = 4 #int 

In [72]:
# Library Time 
import toyplot
import numpy as np
from random import randint
import textwrap as tw
import random

### DNA Generator

In [77]:
# Generate DNA Sequences 
# DNA base randomizer
def dna_base(x):
    return {
        0: 'G',
        1: 'C',
        2: 'A',
        3: 'T'
    }[x]

# generate dna sequences 
def gen_dna(num_dna, min_length, max_length):
    dna_dict = { } # create empty dictionary 
    for i in range(num_dna):
        dna_sequence = ""
        for _ in range(random.randint(min_length, max_length)):
            dna_sequence += dna_base(random.randint(0,3))
        dna_title = ">sequence " + str(i + 1) # make it a FASTA format 
        dna_dict[dna_title] = dna_sequence

    return dna_dict

# function to write generate sequence to a file 
def write_sequence(dictionary, filePath):
    with open (filePath, "w") as f:
        for dna_key, dna_value in dictionary.items():
            f.write(dna_key)
            f.write('\n')
            f.write('\n'.join(tw.wrap(dna_value, 10000))) 
            #f.write('\n')

In [78]:
# Create DNA sequence FASTA file
# Change dna_len, min_length, max_length in first cell!

dna_dict = gen_dna(dna_len, min_length, max_length)
write_sequence(dna_dict, file_name) # write to a fasta file


### De Bruijn Graph

In [111]:
# create De Bruijn network 
# need to make left and right mers and they will serve as nodes
# the edges will be where the kmers overlap with each other 
# we will also create an adjaceny matrix (connection matrix)
# 
# first make the kmers from the input 

def make_kmers(sequence, kmer_size):
    """
    inputs: 
        sequence: dna sequence 
        kmer_size
    """
    kmer_list = []
    num_kmers = len(sequence) - kmer_size + 1

    for i in range(num_kmers):
        seqs = sequence[i:i + kmer_size]
        kmer_list.append(seqs)

    return kmer_list

def de_bruijn(kmer):
    # create empty dictionaries 
    left_mers=[] # left kmer, this will overlap with the right kmer
    right_mers=[] # right kmer, this will overlap with the left kmer
    edges = [] # edge, this corresponds to the overlap between two kmers (1 left kmer and 1 right kmer)
    nodes = set() # nodes will be the kmers

    # first, divide the kmer into left and right mers 
    for i in range(len(kmer)):
        divide = len(kmer[i])
        right_mers.append(kmer[i][1:divide])
        left_mers.append(kmer[i][0:divide-1])
    
    # create the edges and nodes 
    for j in range(len(kmer)):
        edges.append((left_mers[j], right_mers[j])) # creating edge where parts of the left and right mers overlap
        nodes.add(left_mers[j]) # combine the right and left mers
        nodes.add(right_mers[j])

    return right_mers, left_mers, nodes, edges

def plot_debruijn(edges, v):
    
    #returns a toyplot graph from an input of edges
    visualization = toyplot.graph(
        [i[0] for i in edges],
        [i[1] for i in edges],
        width=2000,
        height=700,
        tmarker=toyplot.marker.create(shape='>', mstyle={"fill":'green'}),
        vsize=30,
        vstyle={"stroke":toyplot.color.black, "fill": "black"},
        vlstyle={"font-size": "10px", "fill":"white"},
        estyle={"stroke": toyplot.color.black},
        layout=toyplot.layout.FruchtermanReingold(edges=toyplot.layout.StraightEdges()))
    return visualization

# make finalizing graph structure 
def db_graph(sequence, kmer_len):
    k=(make_kmers(sequence, kmer_len))
    right_mers, left_mers, nodes, edges = de_bruijn(k)

    print("Input Sequence:", data)
    print("Right-mers:", right_mers)
    print("Left-mers:", left_mers)
    print("Nodes:", nodes)
    print("Edges:", edges)
    print("Number of Kmers:", len(k))
    print("Kmer length:", kmer_len)
    print('Sequence length:', len(data))

    # coverage calculation
    # coverage = L*N/G 
    # L = length of kmer 
    # N = number of generated kmers 
    # G = total length of the input sequence 
    coverage = ((kmer_len*len(k))/len(data))
    print("Coverage:", coverage)

    # error rate calculation
    error_calc = (1-(1*10e-2))
    error_rate = len(data)*(error_calc)
    print('Error Rate per Base = 1-(1*10e-2)')
    print("Error Amount for This Sequence:", error_rate)
    
    plot_debruijn(edges, v=float(kmer_len*7))
    

In [112]:
# Now, we can get the output of our De Bruijn graph 
# First, open the file. this is currently set to be the same file that was created in the beginning, but file name can be interchanged
# i.e., file_name to 'sequence1.fa' in place 

with open(file_name) as file:
    next(file) # skips past the first line that says sequence 
    data = file.read()

db_graph(data, kmer_len) # kmer_len is changable in the first cell


Input Sequence: TAATTTTCTATTAGCCTTTAAT
Right-mers: ['AAT', 'ATT', 'TTT', 'TTT', 'TTC', 'TCT', 'CTA', 'TAT', 'ATT', 'TTA', 'TAG', 'AGC', 'GCC', 'CCT', 'CTT', 'TTT', 'TTA', 'TAA', 'AAT']
Left-mers: ['TAA', 'AAT', 'ATT', 'TTT', 'TTT', 'TTC', 'TCT', 'CTA', 'TAT', 'ATT', 'TTA', 'TAG', 'AGC', 'GCC', 'CCT', 'CTT', 'TTT', 'TTA', 'TAA']
Nodes: {'TTA', 'TAG', 'CTT', 'TCT', 'CTA', 'TAA', 'AAT', 'GCC', 'CCT', 'TTC', 'AGC', 'TTT', 'TAT', 'ATT'}
Edges: [('TAA', 'AAT'), ('AAT', 'ATT'), ('ATT', 'TTT'), ('TTT', 'TTT'), ('TTT', 'TTC'), ('TTC', 'TCT'), ('TCT', 'CTA'), ('CTA', 'TAT'), ('TAT', 'ATT'), ('ATT', 'TTA'), ('TTA', 'TAG'), ('TAG', 'AGC'), ('AGC', 'GCC'), ('GCC', 'CCT'), ('CCT', 'CTT'), ('CTT', 'TTT'), ('TTT', 'TTA'), ('TTA', 'TAA'), ('TAA', 'AAT')]
Number of Kmers: 19
Kmer length: 4
Sequence length: 22
Coverage: 3.4545454545454546
Error Rate per Base = 1-(1*10e-2)
Error Amount for This Sequence: 19.8


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="2000.0px" height="700.0px" viewBox="0 0 2000.0 700.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t0a1f8543e02641269c2c5cf78af94a6a"> AAT AGC ATT CCT CTA CTT GCC TAA TAG TAT TCT TTA TTC TTT

In [97]:
# Optional: write to a png/pdg/jpg whatever works 
# in progress, for some reason it won't return the plot in a pdf


k=(make_kmers(sequence, kmer_len))
right_mers, left_mers, nodes, edges = de_bruijn(k)
#graph = plot_debruijn(edges, v=float(kmer_len*7))

visualization = toyplot.graph(
    [i[0] for i in edges],
    [i[1] for i in edges],
    width=2000,
    height=1000,
    tmarker=toyplot.marker.create(shape='>', mstyle={"fill":'green'}),
    vsize=30,
    vstyle={"stroke":toyplot.color.black, "fill": "black"},
    vlstyle={"font-size": "8px", "fill":"white"},
    estyle={"stroke": toyplot.color.black},
    layout=toyplot.layout.FruchtermanReingold(edges=toyplot.layout.StraightEdges()))


with open(file_name) as file:
    next(file) # skips past the first line that says sequence 
    data = file.read()


import toyplot.pdf
toyplot.pdf.render(visualization[0], "figure1.pdf")